In [ ]:
'''

run_sink_impact_assessment.py

    This script damages nodes one-at-a-time (single point failure analysis) to evaluate
    the number of sinks damaged. It is a computationally intensive script.

'''

print('-----------------------------------------------------------------')
print('Executing script: running run_sink_impact_assessment.py')

import pandas as pd
import os
from tqdm import tqdm

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
sys.path.append('../')

from jem.model import jem
from jem.analyse import analyse
from jem.utils import *

print('imported modules')

# define input files
path_to_flows = '../data/csv/generated_nodal_flows.csv'
path_to_nodes = '../data/spatial/infrasim-network/nodes.shp'
path_to_edges = '../data/spatial/infrasim-network/edges.shp'

nodes = gpd.read_file(path_to_nodes)
#nodes_to_attack = nodes.head(5).id.to_list()
nodes_to_attack = nodes.id.to_list()

print('loaded nodes to attack')

results_dataframe = []
count = 1
print('beginning loop...')
for i in range(0,len(nodes_to_attack)):
    print('running iteration ' + str(count) + ' of ' + str(len(nodes_to_attack)))
    # print(count)
    # init model
    run = jem(path_to_nodes,
            path_to_edges,
            path_to_flows,
            #timesteps=1,
            print_to_console=False,
            nodes_to_attack=[nodes_to_attack[i]],
            #edges_to_attack=['edge_25297'],
            super_source=True,
            super_sink=False)
    
    # build model
    run.build()
    
    # run model
    run.optimise(print_to_console=False)
    
    # init results
    results = analyse(model_run=run)
    
    # get results
    nodes_with_shortfall = results.nodes_with_shortfall().node.to_list()
    population = results.get_population_at_nodes(nodes_with_shortfall,col_id='sink_id')
    demand = results.get_demand_at_nodes(nodes_with_shortfall,col_id='sink_id')
    node_types = results.nodes.loc[results.nodes.id.isin(nodes_with_shortfall),'asset_type'].to_list()
    # merge
    df = population.merge(demand,on='sink_id')
    # add node types
    df['sink_type'] = node_types
    # add total number of sinks impacted
    df['total_sinks_impacted'] = df.shape[0]
    # add damaged node id
    df['damaged_node_id'] = nodes_to_attack[i]
    # reindex
    df = df[['damaged_node_id','sink_id','sink_type','total_sinks_impacted','population','demand']]
    # append
    results_dataframe.append(df)
    # append counter
    count = count + 1

print('done!')

print('Saving results: running run_sink_impact_assessment.py')
results_dataframe = pd.concat(results_dataframe,ignore_index=True)
results_dataframe.to_csv('../outputs/sink_impact_assessment.csv',index=False)

print('-----------------------------------------------------------------')

In [ ]:
nodes_to_attack[i]

In [ ]:
results_dataframe

In [ ]:
count = 150

In [ ]:
count % 25